<a href="https://colab.research.google.com/github/LucasJFaust/bibliotecas_py/blob/main/Otimiza%C3%A7%C3%A3o_Hiperpar%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

In [19]:
features = pd.read_excel("temps.xlsx")
features.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [20]:
features.describe()

,year,month,day,temp_2,temp_1,average,actual
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000


In [21]:
features = pd.get_dummies(features)
features.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,0,0,0,0,0,1,0


In [22]:
# Labels are the values we want to predict
labels = np.array(features['actual'])
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('actual', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [23]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [24]:
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn import metrics

## Treino com GradientBoostingRegressor

In [25]:
grb = GradientBoostingRegressor(n_estimators=100)
grb.fit(train_features, train_labels)
gbr_pred = grb.predict(test_features)

In [26]:
# Calculate the absolute errors
errors_gbr = abs(gbr_pred - test_labels)

# Print out the mean absolute error (mae)
r_sq = grb.score(test_features, test_labels)
print('Coeficiente de Determinação (R²):', r_sq)

print('MAE:', metrics.mean_absolute_error(test_labels, gbr_pred))
print('MSE:', metrics.mean_squared_error(test_labels, gbr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, gbr_pred)))

Coeficiente de Determinação (R²): 0.7953242259402815
MAE: 4.0711486946589766
MSE: 28.46402112360264
RMSE: 5.33516833132776


## GridSearch GradientBoosting

In [44]:
from tables.file import parameters
from sklearn.model_selection import GridSearchCV


parameters = {'learning_rate': [0.03],
                  'subsample'    : [0.2],
                  'n_estimators' : [100],
                  'max_depth'    : [8]
                 }

grid_search = GridSearchCV(grb, parameters, scoring='r2', cv=2, n_jobs=-1)

In [45]:
grid_search.fit(train_features, train_labels)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.03], 'max_depth': [8],
                         'n_estimators': [100], 'subsample': [0.2]},
             scoring='r2')

In [46]:
    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_search.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.03, max_depth=8, subsample=0.2)

 The best score across ALL searched params:
 0.8112477345880175

 The best parameters across ALL searched params:
 {'learning_rate': 0.03, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.2}


In [48]:
best_model = grid_search.best_estimator_

In [49]:
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.03,
 'loss': 'squared_error',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 0.2,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [52]:
grb_tunned = GradientBoostingRegressor(alpha = 0.9,  ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init = None,
 learning_rate = 0.03,
 loss = 'squared_error',
 max_depth = 8,
 max_features = None,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 1,
 min_samples_split = 2,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 100,
 n_iter_no_change = None,
 random_state = None,
 subsample = 0.2,
 tol = 0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [53]:
grb_tunned.fit(train_features, train_labels)
gbr_tunned_pred = grb_tunned.predict(test_features)

In [58]:
r_sq = grb_tunned.score(test_features, test_labels)
print("R^2", r_sq)
print("MAE:", metrics.mean_absolute_error(test_labels, grb_tunned.predict(test_features)))

R^2 0.8006452835278162
MAE: 3.968058720300223
